In [1]:
import cv2
import numpy as np
import time

In [2]:
# 初始化摄像头
cap = cv2.VideoCapture(4)

In [3]:
# 设置摄像头的帧大小
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

True

In [4]:
lower_blue = np.array([100, 50, 50])
upper_blue = np.array([130, 255, 255])
lower_red = np.array([0, 50, 50])
upper_red = np.array([10, 255, 255])

In [5]:
start_time = time.time()
while True:
    # 读取一帧图像
    ret, frame = cap.read()

    if not ret:
        break

    # 转换图像到HSV色彩空间
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # 使用颜色阈值找到蓝色目标
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
    _, blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#这_,把两输入改成单输入

    # 寻找蓝色目标的区域
    for contour in blue_contours:
        if cv2.contourArea(contour) > 1000:
            # 找到目标的左上、左下、右上和右下四个点
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int0(box)

            # 绘制目标区域
            cv2.drawContours(frame, [box], 0, (255, 0, 0), 2)
            
            # 找到左上和右下点的坐标
            left_top = tuple(box[box[:, 1].argmin()])
            right_bottom = tuple(box[box[:, 1].argmax()])

            # 在图像上标记左上和右下点的位置
            cv2.circle(frame, left_top, 5, (0, 255, 0), -1)
            cv2.circle(frame, right_bottom, 5, (0, 255, 0), -1)

            # 连接线段并标注交点
            mid_point1 = ((left_top[0] + right_bottom[0]) // 2, left_top[1])
            mid_point2 = ((left_top[0] + right_bottom[0]) // 2, right_bottom[1])
            intersection_point = ((left_top[0] + right_bottom[0]) // 2, (left_top[1] + right_bottom[1]) // 2)

            cv2.line(frame, left_top, right_bottom, (0, 255, 0), 2)
            cv2.line(frame, mid_point1, mid_point2, (0, 255, 0), 2)
            cv2.circle(frame, intersection_point, 5, (0, 0, 255), -1)
    
    # 在图像的左上角显示当前的帧率
    cv2.putText(frame, f"FPS: {int(1 / (time.time() - start_time))}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (0, 255, 0), 2)

    # 显示图像
    cv2.imshow('Frame', frame)

    # 按 'q' 键退出循环
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放摄像头并关闭窗口
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 